### 이전 코드에서 생성한 데이터 불러오기

In [1]:
import numpy as np
# continual learning을 위한 데이터셋

train_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
train_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_A.npy")
train_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_B.npy")
train_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_C.npy")

train_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_A.npy")
train_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_B.npy")
train_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_C.npy")

val_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
val_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_A.npy")
val_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_B.npy")
val_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_C.npy")

val_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_A.npy")
val_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_B.npy")
val_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_C.npy")

test_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
test_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_A.npy")
test_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_B.npy")
test_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_C.npy")

test_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_A.npy")
test_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_B.npy")
test_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_C.npy")


# joint training 을 위한 데이터셋

X_train_3 = np.load("../dataset/joint_training_dataset/X_joint_train_data.npy")
y_train_3 = np.load("../dataset/joint_training_dataset/y_joint_train_data.npy")
X_val_3 = np.load("../dataset/joint_training_dataset/X_joint_val_data.npy")
y_val_3 = np.load("../dataset/joint_training_dataset/y_joint_val_data.npy")
X_test_3 = np.load("../dataset/joint_training_dataset/X_joint_test_data.npy")
y_test_3 = np.load("../dataset/joint_training_dataset/y_joint_test_data.npy")


### 데이터로더 래핑

In [2]:
# 상위 디렉토리로 작업환경 이동
import os
os.chdir(os.path.join(globals()['_dh'][0], ".."))

In [48]:
from dfb.dataset import *
from dfb.processing import *
import experiment
import torchvision.transforms as transforms

model_name = "wdcnn3"

sample_length = experiment.get_sample_length(model_name)
tf_data = experiment.get_transform(model_name)
tf_data = transforms.Compose(tf_data)
tf_label = NpToTensor()
batch_size = 128
num_worker = 4

data_handler = DatasetHandler()

data_handler.assign(
    X_train_3, y_train_3, X_val_3, y_val_3, X_test_3, y_test_3,
    sample_length, "3", tf_data, tf_label, batch_size, num_worker
)

### 모델 가져오기

In [61]:

# 모델 수정되는 경우 해당 셀에서 바로 갱신되도록 autoreload사용
%load_ext autoreload
%autoreload 2

from dfb.model.wdcnn2 import *
from dfb.model.wdcnn3 import *
from dfb.model.wdcnn4 import *
# 클래스 수는 맘대로 지정

# 클래스 수 10개짜리 모델 만들기
model = WDCNN3(n_classes=3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 훈련하기

In [64]:
from dfb.trainmodule import *

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

# ticnn 플래그는 일단 신경쓸 필요 X
training_module = PlModule(model, optimizer, loss, True)

n_steps_d = len(data_handler.dataloaders["3"]["train"].dataset) // (batch_size)

# val_loss 가 가장 작은 모델을 저장하는 콜백
callback = pl.callbacks.ModelCheckpoint(monitor="val_loss",
                                        dirpath=f"./logs/test/best_model",
                                        filename=f"model",
                                        save_top_k=1,
                                        mode="min")
trainer = pl.Trainer(
    gpus = [0],
    max_epochs=100,
    val_check_interval= n_steps_d,
    default_root_dir="./logs/test",
    callbacks=[callback]
)

result = trainer.fit(model=training_module,
                     train_dataloaders=data_handler.dataloaders["3"]["train"],
                     val_dataloaders=data_handler.dataloaders["3"]["val"])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | WDCNN3           | 15.4 K
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
15.4 K    Trainable params
0         Non-trainable params
15.4 K    Total params
0.061     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 24/24 [00:01<00:00, 21.87it/s, loss=0.0121, v_num=74, val_loss=0.896, val_acc=0.838, train_loss=0.0105, train_acc=0.997]   

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 24/24 [00:01<00:00, 21.76it/s, loss=0.0121, v_num=74, val_loss=0.896, val_acc=0.838, train_loss=0.0105, train_acc=0.997]


### 테스트하기

In [66]:
training_module.load_from_checkpoint(f"./logs/test/best_model/model-v27.ckpt",
                                            model=model, optimizer=optimizer,
                                            loss_fn=loss)
result = trainer.test(model=training_module, dataloaders=data_handler.dataloaders["3"]["test"])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 105.50it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9893617033958435
        test_loss           0.0253494493663311
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
